# Predicting parallax observables for NASA Cumulus

This notebook computes the time difference between the peaks of a microlensing event, as observed from two observers. This is defined in S. Calchi Novati, G. Scarpetta 2015 https://arxiv.org/abs/1512.09141 as equation (2):

$$\tau = \frac{t_{0,2} - t_{0,1}}{t_E}$$

Here $t_{0,1}$, $t_{0,2}$ indicate the time at maximum magnification of the microlensing event as seen by the two observers. 

In terms of physical parameters, $\tau$ is expressed as equation (3):

$$\tau = \pi_E (\cos(\chi)\Delta y_0  - \sin(\chi)\Delta x_0 ) $$ 

where $(\Delta x_0,\Delta y_0)$ are the "distances of the observers position projected on the lens plane". $\chi$ is an angle that expressed the angle of the lens motion with respect to the coordinate system. 

## The code pseudocode



### Define the time of observations
### Obtain heliocentric RA/Dec of observers from JPL HORIZONS at a specific time
### Compute the coordinates of each observatory projected onto the observation plane.
#### Set heliocentric coordinates of source star
#### Project Observatory 1 onto observer plane
#### Project Observatory 2 onto observer plane
### Compute Differences In Projected Observatory Coordinates In The Observer Plane
### Generate a lensing system
### Compute 3D and projected distances between observatories
### PROJECT the distances between the two observatories, in both coordinates TO LENS PLANE
### Compute tau and convert




In [2]:
#Install astroquery
!pip install astroquery

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from astropy import constants as const
from astropy import units as u
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from astropy.time import Time
from astroquery.jplhorizons import Horizons

In [ ]:
class observatory:
    """ Class for representing an observatory with heliocentric ra/Dec and range. """

    def __init__(self,position,name):
        
        self.ra = position[0]
        self.dec = position[1]
        self.r = position[2]
        self.name = name

#Define lens system parameters
def lensingsystem():
    #This function sets the following parameters for the lensing system
    #
    # Ds (parsecs) distance to the source star
    # Dl (parsecs) distance to the lens star
    # Ml (solar units) mass of lens object
    # vperp (km/sec) perpendicular relative source-lens velocity
    #
    # In the future, this function will likely be replaced with a function that draws these values from a Galactic model
    Ds = 8000 * u.parsec
    Dl = 6000 * u.parsec
    Ml = 0.3 * u.Msun
    vperp = 200 *u.km/u.s
    x = Dl/Ds             # compute ratio for easier use in 
    
    Re = np.sqrt(4*const.G*Ml/const.c**2 * Ds * x * (1-x))
    te = Re/vperp         # compute Einstein ring radius crossing time
    piE = (1-x)/Re        # definition of parallax from Caldi 
    return [Ds,Dl,x,Ml,Re,te,piE]

#Define pointing vector to source star
def vectorTowardsSourceStar(ra_deg,dec_deg):
    #This function computes the vector pointing towards the source star from the Sun
    #
    #Inputs: ra/dec (degrees) of source star (we assume geocentric equatorial coordinates are the same as helicentric coordinates)
    #
    #Output: a heliocentric 3-vector (unitless)
    #
    
    ra_rad = ra_deg.to('rad')   # convert input ra in degrees to ra in radians
    dec_rad = dec_deg.to('rad') # convert input dec in degrees to dec in radians
    
    n_hat_sourcestar = [np.cos(ra_rad)*np.cos(dec_rad), np.sin(ra_rad)*np.cos(dec_rad), np.sin(dec_rad)]
    
    return n_hat_sourcestar  # this is a unit normal vector


#Define vector to observatory
def vectorTowardsObservatory(ra_deg,dec_deg,r):
    #This function computes the vector pointing towards an observatory from the Sun
    #
    #Inputs: 
    # ra/dec (degrees) of source star (we assume geocentric equatorial coordinates are the same as helicentric coordinates)
    # r (metres) heliocentric distance of observatory
    #
    #Output: a heliocentric 3-vector (meters)
    #
    
    ra_rad = ra_deg.to('rad')   # convert input ra in degrees to ra in radians
    dec_rad = dec_deg.to('rad') # convert input dec in degrees to dec in radians
    
    u_obs = [np.cos(ra_rad)*np.cos(dec_rad), np.sin(ra_rad)*np.cos(dec_rad), np.sin(dec_rad)] * r
    
    return u_obs  # this is a 3-vector with magnitude r


#Project 3-vector onto a plane defined by a normal vector
def projectVectorPlane(u,n):
    #This function projects a 3-vector, u, onto the plane defined by its normal, n.
    #
    #For microlensing, we define the observer plane as the that defined by the vector pointing towards the source star.
    #The position vector u, will be the location in 3-space of an observer.
    #The vector n is normal to the observer plane
    #
    n_norm_sq = np.linalg.norm(n) # Compute the norm of the vector n, squared
    out = u - np.dot(u,n)/n_norm_sq*n # Compute the 3-vector of the projection of u onto the plane normal to n
    
    
    return out 

#Project 3-vector onto coordinate vectors in the observer plane
def projectVectorCoords(ra_deg,dec_deg,u_obs):
    #This function projects the 3-vector u onto the orthogonal vectors lying in the observer plane, which has its normal pointing to ra_deg, dec_deg
    #Inputs
    # ra/dec (degrees) of source star (we assume geocentric equatorial coordinates are the same as helicentric coordinates)
    # u_obs (meters) the heliocentric location in 3-space of an observer. Probably the output from vectorTowardsObservatory()
    #
    # output (metres) : u1, u2: positions in the observer plane of the observatory at u in orthogonal directions n1, n2 (unit 3-vectors) in the observer plane

    ra_rad = ra_deg.to('rad')   # convert input ra in degrees to ra in radians
    dec_rad = dec_deg.to('rad') # convert input dec in degrees to dec in radians
    
    n1 = [-np.sin(ra_rad), np.cos(ra_rad), 0] # This is a unit vector in the observer plane
    n2 = [-np.cos(ra_rad)*np.sin(dec_rad), -np.sin(ra_rad)*np.sin(dec_rad), np.cos(dec_rad)] # This is a unit vector in the observer plane and normal to n2
    
    #the dot product of a vector a with unit vector u is defined to be the projection of a in the direction of u
    u1 = np.dot(u_obs,n1) # Projection of observatory position u, in the direction of unit vector n1
    u2 = np.dot(u_obs,n2) # Projection of observatory position u, in the direction of unit vector n2
    return [u1,u2,n1,n2]


#Return heliocentric RA/Dec for an observer from JPL HORIZONS
def getObserverPostion(idstr,epoch):
    # This function queries the JPL HORIZONS services and returns the heliocentric ra/dec and range
    # for an observer an observatory identified with idstr
    #
    # Inputs:
    #
    # idstr: the JPL HORIZONS identification string for an observer
    # epoch: time in Julian date of the observation
    #
    # Outputs:
    #
    # [RA, Dec, range]: Heliocentric RA/Dec (degrees) and range (AU)
    observer_obj = Horizons(id=idstr,id_type='majorbody',location='500@10',epochs=epoch) # create horizons object for observer, at the body centre of the Sun
    eph = observer_obj.ephemerides(extra_precision=True) #Query HORIZONS for observer
    return [eph['RA'][0]*u.deg, eph['DEC'][0]*u.deg, eph['r'][0]*const.au]

#Compute tau, the difference between microlensing peak times
def computeTau(delta_x,delta_y,piE,chi_deg):
    # This function computes the value tau, which is defined as ( t_{0,2} - t_{0,1} ) / tE, the difference between times of peak magnification of a microlensing event
    # as seen from two observers, scaled by the einstein ring crossing time. 
    #
    # Equation (3) in Calchi Novati and Scarpetta gives the computation of tau in terms of the distance between observer positions projected onto the lens plane, 
    # the parallax magnitude, piE, and a random angle chi which relates the coordinate system to the lens direction of motion. 
    #
    # Inputs:
    #
    # delta_x, delta_y (metres): the distance, in each orthoganal coordinate direction,  between the two observatories, projected onto the lens plane
    #
    # piE : (metres^{-1}): the magnitude of the parallax vector
    #
    # chi: (radians): an angle defining the lens direction of motion to the coordinate system.
    #
    # Outputs:
    #
    # tau: (unitless):  the difference between times of peak magnification of a microlensing event as seen from two observers, scaled by the einstein ring crossing time
    
    chi_rad = chi_deg.to('rad')   # convert input angle chi in degrees to ra in radians
    tau = piE * (np.cos(chi_rad)*delta_y - np.sin(chi_rad)*delta_x)
    return tau


In [ ]:
# NOTE: This cell should be able to be completely commented out 
# 
# It is only intended for unit testing and sanity checks. 

#Unit testing lensingsystem()
[Ds,Dl,x,Ml,Re,te,piE] = lensingsystem()

#Unit testing vectorTowardsSourceStar
n_hat_towardsSourceStar = vectorTowardsSourceStar(170*u.deg,-50*u.deg)
print('The magnitude of the vector pointing to the source star should be 1.0: norm(n_hat_towardsSourceStar) = {}'.format(np.linalg.norm(n_hat_towardsSourceStar) ))

#Unit testing vectorTowardsObservatory
r_unittest = 4e6*u.m
u_obs = vectorTowardsObservatory(12*u.deg,76*u.deg,r_unittest)
print('The magnitude of the vector of the observatory should be {}: norm(u_obs) = {}'.format(r_unittest, np.linalg.norm(u_obs) ))

#Unit testing projectVectorPlane
u_proj = projectVectorPlane(u_obs,n_hat_towardsSourceStar)
print('This is the projection of the observatory position onto the observer plane: {}'. format(u_proj))

#Unit testing projectVectorCoords
[u1,u2,n1,n2] = projectVectorCoords(170*u.deg,-50*u.deg,u_obs)
print('These are the projectons of the observatory position in two orthogonal directions in the observer plane: {}, {}'.format(u1,u2))
print('n1 and n2 are unit vectors. The norms should be 1. norm(n1)={} norm(n2)={}'.format(np.linalg.norm(n1),np.linalg.norm(n2) ))
print('n1 and n2 are orthogonal vectors. Their dot product should be zero. n1 . n2 = {}'.format(np.dot(n1,n2)))
print('u1*n1 and u2*n2 are orthogonal vectors.  Their dot product should be zero. u1*n1 . u2*n2 = {}'.format(np.dot(u1*n1,u2*n2)))
print('n1 and n_hat are orthogonal vectors. Their dot product should be zero. n1 . n_hat ={}'.format(np.dot(n1,n_hat_towardsSourceStar)))
print('n2 and n_hat are orthogonal vectors. Their dot product should be zero. n2 . n_hat ={}'.format(np.dot(n2,n_hat_towardsSourceStar)))

print('The dot product of the projection of u onto the observer plane with n1 should equal u1. u_proj . n1 = {}. u1 = {}'.format(np.dot(u_proj,n1),u1))
print('The dot product of the projection of u onto the observer plane with n2 should equal u2. u_proj . n2 = {}. u2 = {}'.format(np.dot(u_proj,n2),u2))



The magnitude of the vector pointing to the source star should be 1.0: norm(n_hat_towardsSourceStar) = 1.0
The magnitude of the vector of the observatory should be 4000000.0 m: norm(u_obs) = 3999999.9999999995 m
This is the projection of the observatory position onto the observer plane: [-1300613.70183397   597427.61580194  1161814.55849268] m
These are the projectons of the observatory position in two orthogonal directions in the observer plane: -362502.1487332457 m, 1807462.5910403677 m
n1 and n2 are unit vectors. The norms should be 1. norm(n1)=0.9999999999999999 norm(n2)=0.9999999999999999
n1 and n2 are orthogonal vectors. Their dot product should be zero. n1 . n2 = 0.0
u1*n1 and u2*n2 are orthogonal vectors.  Their dot product should be zero. u1*n1 . u2*n2 = 0.0 m2
n1 and n_hat are orthogonal vectors. Their dot product should be zero. n1 . n_hat =0.0
n2 and n_hat are orthogonal vectors. Their dot product should be zero. n2 . n_hat =-5.551115123125783e-17
The dot product of the pro

In [ ]:
# Define the time of observations

epochstr = '2021-04-11T00:00:00' 
t = Time(epochstr, format='isot', scale='utc') #create a astropy time object

In [ ]:
# Obtain heliocentric RA/Dec of observers from JPL HORIZONS at a specific time

obs_Spitzer = observatory(getObserverPostion('spitzer',t.jd),'Spitzer') # Find the position of Spitzer at time t.jd and load into observatory object
obs_Earth = observatory(getObserverPostion('399',t.jd),'Earth')         # Find the position of Earth geocentre at time t.jd and load into observatory object



In [ ]:
# Compute the coordinates of each observatory projected onto the observation plane. 

# Set heliocentric coordinates of source star
ra_source = 175*u.deg
dec_source = 50*u.deg



#Project Observatory 1 onto observer plane
u_obs1 = vectorTowardsObservatory(obs_Earth.ra,obs_Earth.dec,obs_Earth.r)           # compute 3-space heliocentric observatory vector (Observatory 1)
[u11,u12,n11,n12] = projectVectorCoords(ra_source,dec_source,u_obs1)                # project 3-space heliocentric observatory vector onto observer plane (Observatory 1)


#Project Observatory 2 onto observer plane
u_obs2 = vectorTowardsObservatory(obs_Spitzer.ra,obs_Spitzer.dec,obs_Spitzer.r)     # compute 3-space heliocentric observatory vector (Observatory 2)
[u21,u22,n21,n22] = projectVectorCoords(ra_source,dec_source,u_obs2)                # project 3-space heliocentric observatory vector onto observer plane (Observatory 2)


#Sanity check
# The first value in each line below is computed by finding the norm of the heliocentric vector position of the observatory computed from vectorTowardsObservatory
# The second value in each line below is the range value returned directly from JPL Horizons 
#print('The norm of the vector u_obs1 should equal the range to observatory 1. ||u_obs1|| = {}  obs1_r = {}'.format(np.linalg.norm(u_obs1),obs_Earth.r))
#print('The norm of the vector u_obs2 should equal the range to observatory 2. ||u_obs2|| = {}  obs2_r = {}'.format(np.linalg.norm(u_obs2),obs_Spitzer.r))


#Compute Differences In Projected Observatory Coordinates In The Observer Plane
delta_x_observerPlane = u11 - u21  # Notice that "x" and "y" are labels here indicating orthogonal directions. 
delta_y_observerPlane = u12 - u22  


#Generate a lensing system
[Ds,Dl,x,Ml,Re,te,piE] = lensingsystem()  # This is eventually going to be a function that draws systems from a Galactic model. 
print('Source distance and coordinates: Ds = {},  RA = {},  Dec= {}'.format(Ds,ra_source,dec_source))
print('Generated a lens system with lens mass {} at {}. Einstein radius = {} and crossing time = {}'.format(Ml,Dl,Re.to(u.au),te.to(u.day)))
print('Parallax vector magnitude |piE| = {}'.format(piE.decompose()))


#Compute 3D and projected distances between observatories
dist_obs3D = np.linalg.norm(u_obs1 - u_obs2)                                 #This is the distance between the two observatories
dist_obsplane = np.sqrt(delta_x_observerPlane**2 + delta_y_observerPlane**2) # This is the distance between the two observatories in the observer plane
print('Distance between observatories in 3D: {}. Projected onto observer plane: {}'.format(dist_obs3D.to(u.au),dist_obsplane.to(u.au)))

# PROJECT the distances between the two observatories, in both coordinates TO LENS PLANE
delta_x_lensPlane =  delta_x_observerPlane * (1 - x) # Note that x == D_l/D_s
delta_y_lensPlane =  delta_y_observerPlane * (1 - x)


# COMPUTE TAU
chi_deg = 180*u.deg # Angle chi in degrees
tau = computeTau(delta_x_lensPlane,delta_y_lensPlane,piE,chi_deg)
print('tau = {}. Lens direction angle (chi) = {}'.format(tau.decompose(),chi_deg))

# CONVERT TO REAL TIME DIFFERENCE
delta_t = te*tau
print('Real time difference = {}'.format(delta_t.to(u.day)))


Source distance and coordinates: Ds = 8000.0 pc,  RA = 175.0 deg,  Dec= 50.0 deg
Generated a lens system with lens mass 0.3 solMass at 6000.0 pc. Einstein radius = 1.9143494912185357 AU and crossing time = 16.573067573027853 d
Parallax vector magnitude |piE| = 8.729580404377371e-13 1 / m
Distance between observatories in 3D: 1.8615085384001608 AU. Projected onto observer plane: 1.8199987172020968 AU
tau = 0.04207092429734122. Lens direction angle (chi) = 180.0 deg
Real time difference = 0.6972442712395751 d
